In [9]:
import os
from PyPDF2 import PdfReader
import re
import tensorflow as tf
from transformers import BertTokenizer, TFBertForQuestionAnswering
import numpy as np
import requests

ModuleNotFoundError: No module named 'PyPDF2'

In [10]:
!pip install PyPDF2

In [11]:

def extract_text_from_pdf(pdf_path):
    text = ""
    with open(pdf_path, 'rb') as file:
        pdf_reader = PdfReader(file)
        for page in pdf_reader.pages:
            text += page.extract_text()
    return text

In [4]:

def preprocess_text(text):
  
    text = re.sub(r'\n+', ' ', text) 
    text = re.sub(r'\s+', ' ', text)  
    return text

In [5]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = TFBertForQuestionAnswering.from_pretrained('bert-base-uncased')

NameError: name 'BertTokenizer' is not defined

In [12]:
# Function to preprocess the dataset
def preprocess_data(pdf_directory):
    dataset = []
    for filename in os.listdir(pdf_directory):
        if filename.endswith('.pdf'):
            pdf_path = os.path.join(pdf_directory, filename)
            text = extract_text_from_pdf(pdf_path)
            text = preprocess_text(text)
            dataset.append(text)
            print(len(text))
    return dataset

In [13]:
# Preprocess the dataset
pdf_directory = 'datasets'
dataset = preprocess_data(pdf_directory)

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'datasets'

In [14]:
def preprocess_input(input_text):
  
    return input_text.lower()

In [41]:

optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
model.compile(optimizer=optimizer)

In [42]:
def process_large_context(context, question):
    answers = []
    start_logits_list = []
    end_logits_list = []
    window_size = 512 
    stride = 256  
    for i in range(0, len(context), stride):
        window = context[i:i + window_size]
        predicted_answer, start_logits, end_logits = predict_answer(window, question)  # Pass tokenized context to predict_answer
        answers.append(predicted_answer)
        start_logits_list.append(start_logits)
        end_logits_list.append(end_logits)
    return answers, start_logits_list, end_logits_list

In [61]:
def predict_answer_for_large_context(context, question):
    context = preprocess_text(context)
    answers, start_logits_list, end_logits_list = process_large_context(context, question)
    answers_list = []
    start_probs_list = []
    end_probs_list = []

  
    for i in range(len(answers)):
        if(answers[i].strip() != ""):
            answers_list.append(answers[i])
            start_probs_list.append(start_logits_list[i])
            end_probs_list.append(end_logits_list[i])
    answers = answers_list

    # Calculate confidence scores for each answer
    confidence_scores = []
    for start_probs, end_probs in zip(start_probs_list, end_probs_list):
        confidence_score = np.sum(start_probs) * np.sum(end_probs)
        confidence_scores.append(confidence_score)

    # Select the answer with the highest confidence score
    best_answer_index = np.argmax(confidence_scores)
    best_answer = answers[best_answer_index]
    best_confidence_score = confidence_scores[best_answer_index]

    return best_answer, best_confidence_score

In [62]:

def predict_answer(context, question):
  
    tokenized_input = tokenizer(context, question, padding='max_length', truncation='only_first', max_length=512, return_tensors="tf")
    model_trained = model(tokenized_input)

    start_logits = model_trained[0]
    end_logits = model_trained[1]
    
    start_logits = start_logits[0]
    end_logits = end_logits[0]

    start_logits = np.array(start_logits).astype(np.float32)
    end_logits = np.array(end_logits).astype(np.float32)
    
    start_index = np.argmax(start_logits)
    end_index = np.argmax(end_logits)
    
 
    answer_tokens = tokenized_input["input_ids"][0][start_index:end_index+1].numpy()
    
    # Decode the answer tokens into text using the tokenizer
    answer = tokenizer.decode(answer_tokens)
    
    # Return the decoded answer
    return answer, start_logits, end_logits

In [63]:
# Function to start the chatbot
def chatbot():
    print("Chatbot: Hi! I'm here to answer your questions.")
    print("Chatbot: Type 'exit' to end the conversation.")
    while True:
        user_input = input("You: ")
        user_input = preprocess_input(user_input)
        
        if user_input == 'exit':
            print("Chatbot: Goodbye!")
            break
        
        context = dataset[1]
        # print(context)
        predicted_answer, confidence = predict_answer_for_large_context(context, user_input)
        print(confidence)
        print("Chatbot:", predicted_answer)
        break 

In [64]:

chatbot()

Chatbot: Hi! I'm here to answer your questions.
Chatbot: Type 'exit' to end the conversation.


You:  what are solutions


6778.5913
Chatbot: a will be xa = a a b n n n ( 1. 5 ) for a solution containing i number of components, we have : xi = i 1 2 i....... n n n n = i in n ( 1. 6 ) it can be shown that in a given solution sum of all the mole fractions is unity, i. e. x1 + x2 +.................. + xi = 1 ( 1. 7 ) mole fraction unit is very useful in relating some physical properties of solutions, say vapour pressure with the concentration of the
